In [7]:
import sqlite3
from datetime import datetime
from typing import List, Tuple, Optional
import timeit

In [9]:
class Database:
    def __init__(self, db_name: str = '.db\\scripts_database.db'):
        self.db_name = db_name
    
    def create_table(self, table_name, columns: dict = None):
        if columns is None:
            ## Create blank table with autoincrementing primary key 
            columns = {
                "id": "INTEGER PRIMARY KEY AUTOINCREMENT",
            }

        ## Create SQL query string based on dictionary
        columns_sql = ",\n                    ".join(
                    [f"{col} {col_type}" for col, col_type in columns.items()]
                    )

        create_sql = f'''
                    CREATE TABLE IF NOT EXISTS {table_name} (
                        {columns_sql}
                    )
                    '''

        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute(create_sql)
            conn.commit()

    def view_table(self, table_name):
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute(f'SELECT * FROM {table_name}')

            ## Notes -> 7 elements in tuple from c.description provides column metadata (name, type_code, display_size, internal_size, precision, scale, null_ok). 
            print(cursor.description)
            all_rows = cursor.fetchall()
            print(all_rows)

    def insert_into_table(self, table_name, data_dict):
        """
        data_dict: {
            "col1": [val1_row1, val1_row2, ...],
            "col2": [val2_row1, val2_row2, ...],
            ...
        }
        """
        if not data_dict:
            raise ValueError("No data provided")

        columns = list(data_dict.keys())

        ## Next grabs first list from iterator, iter iterates over data_dict values
        num_rows = len(next(iter(data_dict.values())))
        
        ## Validate all columns have the same number of rows
        for col, values in data_dict.items():
            if len(values) != num_rows:
                raise ValueError(f"Column '{col}' has inconsistent number of values")
        
        ## Build list of row tuples
        rows = []
        for i in range(num_rows):
            row = tuple(data_dict[col][i] for col in columns)
            rows.append(row)
        
        # Create SQL with dynamic columns and placeholders
        cols_sql = ", ".join(columns)
        placeholders = ", ".join(["?"] * len(columns))
        insert_sql = f"INSERT INTO {table_name} ({cols_sql}) VALUES ({placeholders})"

        # Insert
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.executemany(insert_sql, rows)
            conn.commit()


def insert_into_table_2(self, table_name, data_dict):
    """
    data_dict: {
        "col1": [val1_row1, val1_row2, ...],
        "col2": [val2_row1, val2_row2, ...],
        ...
    }
    """
    if not data_dict:
        raise ValueError("No data provided")

    columns = list(data_dict.keys())
    
    # Validate all columns have the same number of rows
    lengths = [len(values) for values in data_dict.values()]
    if not all(length == lengths[0] for length in lengths):
        raise ValueError("All columns must have the same number of values")
    
    # Transpose data to create rows using zip
    rows = list(zip(*data_dict.values()))
    
    # Create SQL with dynamic columns and placeholders
    cols_sql = ", ".join(columns)
    placeholders = ", ".join(["?"] * len(columns))
    insert_sql = f"INSERT INTO {table_name} ({cols_sql}) VALUES ({placeholders})"

    # Insert
    with sqlite3.connect(self.db_name) as conn:
        cursor = conn.cursor()
        cursor.executemany(insert_sql, rows)
        conn.commit()

In [4]:
db = Database()

db.view_table('scripts')
db.view_table('clients')
db.view_table('jobs')



OperationalError: unable to open database file

In [ ]:
db.insert_into_table(table_name='clients',
                        data_dict = {"name": ["All American Auto", "Bath, Bed & Beyond"],
                                     "code": ["AAA", "BBB" ],
                                     "description":["Test client 1", "Test client 2"]
                                     }
                        )
